# Kedi ve Köpek Sınıflandırması CNN Projesi – VGG16 Transfer Öğrenimi + Grad-CAM + Random Search + L2 Düzenleme
## 1. Çalışma Ortamının Hazırlanması

Bu kısımda, proje süresince kullanacağımız tüm kütüphaneler yüklenerek çalışma ortamı hazırlanır. Amaç, veri işleme, görselleştirme, model oluşturma ve performans değerlendirmesi için gerekli araçları hazır hale getirmektir.

---

### 🔹 Temel Kütüphaneler
- **NumPy (`np`)** → Sayısal hesaplamalar, dizi ve matris işlemleri için.  
- **Matplotlib & Seaborn** → Eğitim sırasında doğruluk ve kayıp grafikleri ile Confusion Matrix görselleştirmeleri.  
- **os & zipfile** → Dosya ve klasör yönetimi, sıkıştırılmış dosyaların açılması.  
- **cv2 (OpenCV)** → Görsellerin okunması, yeniden boyutlandırılması ve ön işleme.  
- **tqdm** → Döngülerde ilerleme durumunu göstermek için.  
- **random** → Rastgele seçimler, örneğin rastgele görsel göstermek için.  
- **warnings.filterwarnings('ignore')** → Gereksiz uyarıları gizleyerek temiz bir çıktı sağlar.

---

### 🔹 Scikit-learn Araçları
- **train_test_split** → Veri setini eğitim, doğrulama ve test kümelerine böler.  
- **classification_report & confusion_matrix** → Modelin performansını ölçmek için detaylı metrikler sunar.

---

### 🔹 TensorFlow / Keras Bileşenleri
- **Model, Layers (Input, Dense, Flatten, Dropout)** → CNN yapısını kurmak için gerekli katmanlar.  
- **ImageDataGenerator** → Veri artırma teknikleri (döndürme, yakınlaştırma, yatay/vertical çevirme).  
- **to_categorical** → Etiketleri one-hot formatına çevirir.  
- **Callbacks**:
  - `EarlyStopping`: Overfitting önlemek için eğitimi durdurur.  
  - `ModelCheckpoint`: En iyi model ağı kaydeder.  
  - `ReduceLROnPlateau`: Öğrenme oranını otomatik olarak ayarlar.  
- **VGG16 & preprocess_input** → Önceden eğitilmiş model ve uygun giriş verisi ön işleme.  
- **l2** → Ağırlıklara ceza ekleyerek aşırı uyumu (overfitting) azaltır.

---

### 🔹 TensorFlow Sürümü Kontrolü
Ortamda yüklü TensorFlow sürümünü görüntülemek için:
```python
print("TensorFlow sürümü:", tf.__version__)


In [ ]:
# 1. Ortam Kurulumu
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import zipfile
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.regularizers import l2

print("TensorFlow version:", tf.__version__)

## 2. Veri Yükleme ve Eğitim-Doğrulama-Test Bölünmesi

Bu bölümde, Dogs vs Cats veri seti açılır, görseller işlenir, etiketlenir ve eğitim süreci için **eğitim, doğrulama ve test** kümelerine ayrılır.

---

### 🔹 Veri Setini Açma
- Kaggle ortamındaki `train.zip` dosyası `/kaggle/working/` dizinine çıkarılır.  
- Çıkarılan dosyaların yolu `rec_egitim_yolu` değişkeninde saklanır.

---

### 🔹 Veri Yükleme Fonksiyonu
- `veri_yukle` fonksiyonu şu işlemleri yapar:  
  - Köpek görselleri → etiket **1**  
  - Kedi görselleri → etiket **0**  
- Her bir görsel:  
  - `cv2.imread` ile okunur  
  - `(128,128)` boyutuna yeniden boyutlandırılır  
  - `preprocess_input` ile normalize edilir  
- Fonksiyonun çıktıları:  
  - `X` → Görsellerin NumPy dizisi  
  - `y` → Etiketler

---

### 🔹 One-Hot Encoding
- `to_categorical` kullanılarak etiketler `[0,1]` veya `[1,0]` formatına çevrilir.  
- Bu dönüşüm, `categorical_crossentropy` kayıp fonksiyonu ile uyumlu çalışır.

---

### 🔹 Sınıf Dağılımı Kontrolü
- `sns.barplot` ile kedi ve köpek görsellerinin sayısı görselleştirilir.  
- Bu adım, veri setinde sınıf dengesinin korunup korunmadığını kontrol etmek için önemlidir.

---

### 🔹 Örnek Görsellerin Gösterimi
- Rastgele seçilen bir kedi ve bir köpek görseli ekrana basılır.  
- Böylece veri yükleme ve ön işleme sürecinin doğru çalıştığı doğrulanır.

---

### 🔹 Eğitim-Doğrulama-Test Ayrımı
- `train_test_split` ile veri önce **%80 eğitim / %20 test** olacak şekilde bölünür.  
- Eğitim verisinin **%20’si** ayrıca doğrulama kümesi olarak ayrılır.  
- `stratify` parametresi sayesinde kedi ve köpek oranı tüm alt kümelerde korunur.  
- Sonuç olarak üç alt küme oluşur:  
  - **Eğitim (Train)** → Modelin öğrenmesi için  
  - **Doğrulama (Validation)** → Hiperparametre ayarlamaları ve overfitting kontrolü için  
  - **Test (Test)** → Nihai model değerlendirmesi için

---

✅ **Özet:**  
Bu aşamada veri seti başarıyla yüklenmiş, işlenmiş, sınıf dağılımı kontrol edilmiş ve görseller **eğitim, doğrulama ve test** kümeleri halinde model eğitimi için hazır hâle getirilmiştir.


In [ ]:
# 2. Veri Yükleme ve Train-Validation-Test Split
ort_veri_yolu = "/kaggle/input/dogs-vs-cats"
with zipfile.ZipFile(os.path.join(ort_veri_yolu, "train.zip"), 'r') as zip_ref:
    zip_ref.extractall("/kaggle/working/")
rec_egitim_yolu = "/kaggle/working/train"

def veri_yukle(veri_yolu, boyut=(128,128)):
    goruntuler, etiketler = [], []
    kopek_yollari = [os.path.join(veri_yolu,f) for f in os.listdir(veri_yolu) if f.startswith('dog')]
    for yol in tqdm(kopek_yollari, desc="Köpekler"):
        img = cv2.imread(yol)
        if img is not None:
            img = cv2.resize(img, boyut)
            img = preprocess_input(img)
            goruntuler.append(img)
            etiketler.append(1)
    kedi_yollari = [os.path.join(veri_yolu,f) for f in os.listdir(veri_yolu) if f.startswith('cat')]
    for yol in tqdm(kedi_yollari, desc="Kediler"):
        img = cv2.imread(yol)
        if img is not None:
            img = cv2.resize(img, boyut)
            img = preprocess_input(img)
            goruntuler.append(img)
            etiketler.append(0)
    return np.array(goruntuler), np.array(etiketler)

X, y = veri_yukle(rec_egitim_yolu)
y_cat = to_categorical(y, 2)

# Sınıf dağılımı görselleştirme
unique, counts = np.unique(y, return_counts=True)
plt.figure(figsize=(6,4))
sns.barplot(x=['Kedi','Köpek'], y=counts)
plt.title("Sınıf Dağılımı")
plt.ylabel("Görüntü Sayısı")
plt.show()

# Örnek görseller
plt.figure(figsize=(12,6))
for i, label in enumerate([0,1]):  # 0=Kedi, 1=Köpek
    idx = np.where(y==label)[0]
    sample_idx = random.choice(idx)
    img = ((X[sample_idx]+1)*127.5).astype('uint8')
    plt.subplot(1,2,i+1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Kedi' if label==0 else 'Köpek')
plt.show()

# Train-Test-Validation split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=np.argmax(y_train, axis=1), random_state=42
)
print(f"Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")

## 3. Veri Artırma (Data Augmentation)

Bu bölümde, eğitim verisinin çeşitliliğini artırmak ve modelin **overfitting** yapmasını engellemek için **veri artırma teknikleri** uygulanır.

---

### 🔹 Veri Artırmanın Amaçları
- Eğitim sırasında verilerden farklı varyasyonlar oluşturur.  
- Modelin genelleme yeteneğini güçlendirir.  
- Özellikle küçük veri setlerinde başarıyı artırır.

---

### 🔹 Kullanılan Parametreler
- **rotation_range=40** → Görselleri ±40 derece arasında rastgele döndürür.  
- **width_shift_range=0.3** → Görselleri yatay eksende %30’a kadar kaydırır.  
- **height_shift_range=0.3** → Görselleri dikey eksende %30’a kadar kaydırır.  
- **horizontal_flip=True** → Görselleri yatayda çevirir (ayna etkisi).  
- **zoom_range=0.3** → Görselleri %30 oranında yakınlaştırır veya uzaklaştırır.  
- **brightness_range=[0.7,1.3]** → Görsellerin parlaklığını %70–%130 aralığında değiştirir.

---

✅ **Özet:**  
Bu aşamada `ImageDataGenerator` ile veriler üzerinde döndürme, kaydırma, yakınlaştırma, yatay çevirme ve parlaklık ayarlamaları uygulanır. Böylece model, farklı görsel varyasyonlarıyla eğitilir ve daha güçlü bir genelleme yeteneği kazanır.


In [ ]:
# 3. Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    zoom_range=0.3,
    brightness_range=[0.7,1.3]
)

## 4. Random Search ile Hiperparametre Denemeleri

Bu bölümde, farklı **hiperparametre kombinasyonları** test edilerek modelin en iyi performansı elde etmesi hedeflenir. Yaklaşım, **Random Search** mantığına dayanmaktadır.

---

### 🔹 Amaç
- Öğrenme oranı (learning rate), dropout oranı ve batch size gibi hiperparametrelerin farklı değerlerini deneyerek modelin performansını değerlendirmek.  
- Validation (doğrulama) başarısını en yüksek seviyeye çıkaran parametre setini belirlemek.

---

### 🔹 İşlem Adımları

1. **Parametre Kombinasyonları**
   - Üç farklı set oluşturulmuştur:  
     - `{learning_rate: 0.001, dropout_rate: 0.5, batch_size: 32}`  
     - `{learning_rate: 0.0005, dropout_rate: 0.5, batch_size: 32}`  
     - `{learning_rate: 0.0001, dropout_rate: 0.4, batch_size: 64}`

2. **Model Kurulumu (VGG16 Transfer Learning)**
   - `VGG16` tabanlı önceden eğitilmiş model (`imagenet`) kullanılır.  
   - `include_top=False` → Son sınıflandırma katmanı çıkarılır.  
   - Tüm katmanlar **dondurulur** (`trainable=False`) ve özellik çıkarıcı olarak kullanılır.

3. **Yeni Katmanların Eklenmesi**
   - `Flatten` → Özellikleri tek boyuta indirger.  
   - `Dense(512, relu)` → 512 nöronlu tam bağlı katman + **L2 regularization**.  
   - `Dropout` → Parametre setindeki değere göre uygulanır.  
   - `Dense(2, softmax)` → Çıkış katmanı (kedi ve köpek sınıfları).

4. **Modelin Derlenmesi**
   - Optimizasyon: **Adam**  
   - Learning rate parametre setinden alınır  
   - Kayıp fonksiyonu: **categorical_crossentropy**  
   - Ölçüt: **accuracy**

5. **Eğitim Süreci**
   - `ImageDataGenerator` ile artırılmış eğitim verisi kullanılır.  
   - `batch_size` parametre setinden alınır.  
   - 5 epoch boyunca eğitim yapılır.  
   - Validation seti ile doğruluk takip edilir.

6. **Sonuçların Karşılaştırılması**
   - Her denemenin en iyi **validation accuracy** kaydedilir.  
   - En yüksek doğruluk sağlayan parametre seti `best_params` olarak seçilir.

---

✅ **Özet:**  
Bu adımda farklı **learning rate, dropout ve batch size** kombinasyonları test edilerek, modelin doğrulama başarısını en üst düzeye çıkaran parametreler belirlenmiştir. Bu yöntem, **hiperparametre optimizasyonu** ile model performansını artırmayı sağlar.


In [ ]:
# 4. Random Search Hiperparametre Denemeleri
param_combinations = [
    {'learning_rate':0.001,'dropout_rate':0.5,'batch_size':32},
    {'learning_rate':0.0005,'dropout_rate':0.5,'batch_size':32},
    {'learning_rate':0.0001,'dropout_rate':0.4,'batch_size':64},
]

best_score = 0
best_params = None

for i, params in enumerate(param_combinations):
    print(f"\nTest Kombinasyonu {i+1}/{len(param_combinations)}: {params}")
    
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128,128,3))
    for layer in base_model.layers:
        layer.trainable = False
    
    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = Dropout(params['dropout_rate'])(x)
    outputs = Dense(2, activation='softmax')(x)
    
    model = Model(base_model.input, outputs)
    opt = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=params['batch_size']),
        steps_per_epoch=len(X_train)//params['batch_size'],
        epochs=5,
        validation_data=(X_val, y_val),
        verbose=1
    )
    
    val_acc = max(history.history['val_accuracy'])
    print(f"En iyi validation accuracy: {val_acc:.4f}")
    
    if val_acc > best_score:
        best_score = val_acc
        best_params = params

print(f"\nEn iyi parametreler: {best_params}, Validation Accuracy: {best_score:.4f}")

## 5. Nihai Model Eğitimi

Bu bölümde, **en iyi hiperparametreler** kullanılarak modelin son eğitimi gerçekleştirilir. Eğitim sırasında **callbacks** ile overfitting önlenir ve en iyi ağırlıklar saklanır.

---

### 🔹 Model Kurulumu
- `VGG16` tabanlı transfer öğrenimi modeli kullanılır.  
- `weights='imagenet'` → Önceden eğitilmiş ağırlıklar yüklenir.  
- `include_top=False` → Son sınıflandırma katmanı çıkarılır.  
- Katmanlar **dondurulur** (`trainable=False`) → Önceden öğrenilmiş özellikler korunur.

---

### 🔹 Yeni Katmanların Eklenmesi
- `Flatten` → Özellikleri tek boyutlu hâle getirir.  
- `Dense(512, relu)` → 512 nöronlu tam bağlı katman + **L2 regularization**.  
- `Dropout` → Hiperparametre optimizasyonunda belirlenen oran uygulanır.  
- `Dense(2, softmax)` → Çıkış katmanı (kedi ve köpek sınıfları).

---

### 🔹 Modelin Derlenmesi
- Optimizasyon: **Adam** ve **en uygun learning rate** kullanılır.  
- Kayıp fonksiyonu: **categorical_crossentropy**  
- Ölçüt: **accuracy**

---

### 🔹 Callbacks Kullanımı
- `EarlyStopping` → Validation kaybı 7 epoch boyunca iyileşmezse eğitimi durdurur ve en iyi ağırlıkları geri yükler.  
- `ModelCheckpoint` → En iyi ağırlıkları `best_model.h5` dosyasına kaydeder.  
- `ReduceLROnPlateau` → Validation kaybı 3 epoch boyunca iyileşmezse öğrenme oranını 0.2 faktörü ile azaltır.

---

### 🔹 Model Eğitimi
- Eğitim, `ImageDataGenerator` ile artırılmış veriler üzerinde yapılır.  
- `batch_size` ve diğer hiperparametreler **en iyi parametreler** setinden alınır.  
- 25 epoch boyunca eğitim gerçekleştirilir ve validation seti ile performans izlenir.

---

✅ **Özet:**  
Bu aşamada, transfer öğrenimli CNN modeli **en iyi hiperparametreler** ile eğitilmiş, overfitting kontrol edilmiştir ve en iyi ağırlıklar kaydedilmiştir. Model artık **test ve değerlendirme** aşamasına hazırdır.


In [ ]:
# 5. Final Model Eğitimi
final_base = VGG16(weights='imagenet', include_top=False, input_shape=(128,128,3))
for layer in final_base.layers:
    layer.trainable = False

x = Flatten()(final_base.output)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(best_params['dropout_rate'])(x)
outputs = Dense(2, activation='softmax')(x)
final_model = Model(final_base.input, outputs)

final_model.compile(
    optimizer=tf.keras.optimizers.Adam(best_params['learning_rate']),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(patience=7, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True),
    ReduceLROnPlateau(patience=3, factor=0.2)
]

history = final_model.fit(
    datagen.flow(X_train, y_train, batch_size=best_params['batch_size']),
    steps_per_epoch=len(X_train)//best_params['batch_size'],
    epochs=25,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

## 6. Model Değerlendirme ve Overfitting/Underfitting Analizi

Bu bölümde, eğitilen modelin performansı görselleştirilir, overfitting veya underfitting olup olmadığı incelenir ve test verisi üzerinden değerlendirme yapılır.

---

### 🔹 Eğitim ve Doğrulama Grafikleri
- `history.history` kullanılarak:  
  - **Loss** (Eğitim ve Validation) grafiği çizilir  
  - **Accuracy** (Eğitim ve Validation) grafiği çizilir  
- Bu grafikler, modelin öğrenme sürecini ve performans eğilimlerini görselleştirir.

---

### 🔹 Overfitting / Underfitting Kontrolü
- Eğitim ve doğrulama accuracy farkı hesaplanır:  
  ```python
  acc_diff = abs(val_acc - train_acc)

---

### 🔹 Durum Değerlendirmesi

- `acc_diff < 0.05` → ✅ Model dengeli, eğitim ve doğrulama performansı birbirine yakın.  
- `val_acc < train_acc` → ⚠️ Bir miktar overfitting olabilir.  
- `val_acc > train_acc` → ⚠️ Bir miktar underfitting olabilir.

---

### 🔹 Test Verisi Üzerinde Performans

- `final_model.evaluate` ile test kaybı ve doğruluk ölçülür.  
- `final_model.predict` ile tahminler alınır ve gerçek sınıflarla karşılaştırılır.

---

### 🔹 Confusion Matrix ve Sınıf Bazlı Raporlama

- `confusion_matrix` kullanılarak doğru ve yanlış sınıflandırmalar **heatmap** ile görselleştirilir.  
- `classification_report` ile her sınıf için:  
  - Precision  
  - Recall  
  - F1-score  
  raporlanır (Kedi ve Köpek sınıfları).

---

✅ **Özet:**  
Bu aşamada modelin eğitim süreci grafiklerle analiz edilmiş, overfitting/underfitting durumu kontrol edilmiş ve test seti üzerinde performansı ölçülmüştür. Confusion matrix ve sınıf bazlı raporlar, modelin hangi sınıflarda daha başarılı olduğunu gösterir.


In [ ]:
# 6. Model Değerlendirme ve Overfitting/Underfitting Kontrolü
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Eğitim Loss')
plt.plot(history.history['val_loss'], label='Doğrulama Loss')
plt.title('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='Eğitim Accuracy')
plt.plot(history.history['val_accuracy'], label='Doğrulama Accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

# Gerçekçi overfit/underfit yorumlama
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]

acc_diff = abs(val_acc - train_acc)

if acc_diff < 0.06:
    print(f"✅ Model dengeli. Eğitim ve doğrulama accuracy farkı çok küçük ({acc_diff:.4f}).")
elif val_acc < train_acc:
    print(f"⚠️ Model biraz overfitting yapıyor olabilir. Accuracy farkı: {acc_diff:.4f}")
else:
    print(f"⚠️ Model biraz underfitting yapıyor olabilir. Accuracy farkı: {acc_diff:.4f}")

test_loss, test_acc = final_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc*100:.2f}%")

y_pred = np.argmax(final_model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

plt.figure(figsize=(6,4))
sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

print(classification_report(y_true, y_pred, target_names=['Kedi','Köpek']))

## 7. Grad-CAM ile Görselleştirme

Bu bölümde, eğitilen CNN modelinin **hangi bölgeleri kullanarak sınıflandırma yaptığı** görselleştirilir. Bu yöntem, modelin karar mekanizmasını anlamak ve yorumlamak için kullanılır.

---

### 🔹 Amaç
- Modelin odaklandığı alanları görselleştirmek.  
- Sınıflandırma kararlarının **görsel olarak açıklanabilirliğini** artırmak.  
- Overfitting veya hatalı sınıflandırma durumlarını incelemek.

---

### 🔹 Grad-CAM Fonksiyonları

1. **make_gradcam_heatmap**  
   - Son Conv2D katmanı ve model çıktısı kullanılarak **gradient tabanlı ısı haritası (heatmap)** oluşturur.  
   - `pred_index` belirtilmezse, modelin en yüksek tahmine göre sınıf seçilir.  
   - Heatmap, ilgili sınıfa ait önem derecelerini 0-1 arasında normalize eder.

2. **display_gradcam**  
   - Heatmap’i orijinal görselin üzerine uygular.  
   - `alpha` parametresi ile ısı haritasının opaklığı ayarlanır.  
   - Sonuç, modelin dikkat ettiği bölgelerin görsel olarak anlaşılmasını sağlar.

---

### 🔹 Görselleştirme Adımları
- Test setinden rastgele 4 görsel seçilir.  
- Her görsel için:  
  - Son Conv2D katmanı belirlenir.  
  - Grad-CAM ısı haritası oluşturulur.  
  - Orijinal görsel ve heatmap birleştirilerek ekrana çizilir.  
- Başlıkta modelin tahmini sınıf (`Pred`) gösterilir.

---

✅ **Özet:**  
Bu aşamada Grad-CAM yöntemi kullanılarak modelin **karar verdiği bölgeler** görselleştirilmiş, modelin “neden bu sınıfı seçtiği” görsel olarak analiz edilmiştir. Böylece modelin açıklanabilirliği ve yorumlanabilirliği artırılmıştır.


In [ ]:
# 7. Grad-CAM
from tensorflow.keras.layers import Conv2D

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = Model(inputs=model.input,
                       outputs=[model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap,0)/tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img, heatmap, alpha=0.4):
    heatmap_resized = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap_colored = cv2.applyColorMap(np.uint8(255*heatmap_resized), cv2.COLORMAP_JET)
    img_original = ((img + 1) * 127.5).astype('uint8')
    superimposed_img = cv2.addWeighted(img_original, 1-alpha, heatmap_colored, alpha,0)
    return superimposed_img

plt.figure(figsize=(12,6))
for i in range(4):
    plt.subplot(2,4,i+1)
    img = np.expand_dims(X_test[i], axis=0)
    last_conv_layer = [layer.name for layer in final_model.layers if isinstance(layer, Conv2D)][-1]
    heatmap = make_gradcam_heatmap(img, final_model, last_conv_layer)
    superimposed = display_gradcam(X_test[i], heatmap)
    plt.imshow(cv2.cvtColor(superimposed, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f'Pred: {y_pred[i]}')
plt.show()

## 8. Modelin Kaydedilmesi

Bu bölümde, eğitilen model **kalıcı olarak saklanır** ve ileride tekrar kullanılmak üzere hazır hâle getirilir.

---

### 🔹 Amaç
- Modeli yeniden eğitmeye gerek kalmadan tekrar kullanabilmek.  
- Test, deployment veya uygulama aşamalarında kolaylık sağlamak.

---

### 🔹 İşleyiş
- `final_model.save("kedi_kopek_model_final.h5")`  
  - Modelin ağırlıkları, yapısı ve eğitim bilgileri `.h5` formatında kaydedilir.  
- `print` ile modelin başarıyla kaydedildiği kullanıcıya bildirilir.

---

✅ **Özet:**  
Bu aşamada, eğitilen CNN modeli `.h5` formatında saklanmış ve artık **kullanıma hazır** hâle gelmiştir. Model, ileride tekrar yüklenip test edilebilir veya uygulamalarda kullanılabilir.


In [ ]:
final_model.save("kedi_kopek_model_final.h5")
print("Model başarıyla kaydedildi: kedi_kopek_model_final.h5")
